<a href="https://colab.research.google.com/github/tuanymariiah/data_analysis/blob/main/Consumo_de_cerveja.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

A cerveja é uma das bebidas mais consumidas no Brasil e no mundo, estando sempre presentes em confraternizações, happy hour, e em diferentes situações.
Mas, qual o país que mais consome cerveja no mundo? E a nível de Brasil, em qual ano houve um aumento considerável no consumo desta bebida?
É comum acharmos que a Alemanha é o país que mais consome cerveja no mundo, devido ao seu surgimento, mas infelizmente não é. A China é o país que mais consome com 27%, seguido pelos Estados Unidos com 13% e em 3º lugar está o Brasil com 7%. 
No Brasil, um dos dos estados que consome cerveja é o estado de São Paulo com 37%, ocupando o primeiro lugar do ranking nacional. O número de vendas de cerveja no Brasil em 2020 foi maior do que em  2014, quando ocorreu a copa do mundo. 

O dataset foi coletado no período de um ano, 01/01/2015 a 31/05/2015, no estado de São Paulo, em uma área universitária, um local onde ocorrem festas com alunos entre 18 a 28 anos. Este dataset pode ser encontrado no kaggle.

In [24]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet, LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
sns.set()


In [25]:
file = '/content/drive/MyDrive/Projetos Pessoais/Portifólio/P[03] - consumo de cerveja -sao paulo/Consumo_cerveja.csv'
#file = 'https://raw.githubusercontent.com/tuanymariiah/data_analysis/main/Bases/Consumo_cerveja.csv?token=AG2B7L7OXCU33OBQDRA36C3BCFIG4'
base_beer = pd.read_csv(file)

Este dataset possui os seguintes atributos
* Data - data da coleta                    
* Temperatura Media (C) - temperatura média daquele dia  
* Temperatura Minima (C) - temperatura mínima daquele dia
* Temperatura Maxima (C) - temperatura máxima daquele dia        
* Precipitacao (mm) - quanto que choveu em milímetros
* Final de Semana - marca 0 ou 1 se aquela data cai em um final de semana               
* Consumo de cerveja (litros) - variável de saída

verificacao de valores ausentes


In [26]:
base_beer.isna().sum()

Data                           576
Temperatura Media (C)          576
Temperatura Minima (C)         576
Temperatura Maxima (C)         576
Precipitacao (mm)              576
Final de Semana                576
Consumo de cerveja (litros)    576
dtype: int64

exclusao dos valores ausentes

In [27]:
base_beer=base_beer.dropna()
base_beer.isna().sum()

Data                           0
Temperatura Media (C)          0
Temperatura Minima (C)         0
Temperatura Maxima (C)         0
Precipitacao (mm)              0
Final de Semana                0
Consumo de cerveja (litros)    0
dtype: int64

Para uma análise exploratória dos dados tracei algumas coisas interessantes a se observar:
* Qual dia da semana houve um consumo maior?
* Qual data houve um consumo maior?
* Qual dia da semana houve um consumo menor?
* Qual data houve um consumo menor?
* Era um dia útil, final de semana ou feriado quando ocorreu um consumo maior?
* Em qual mês houve um consumo maior ?
* Em qual mês houve um consumo menor ?
* Em qual temperatura houve um consumo maior de temperatura?
* A correlacao do consumo de cerveja e os demais atributos

Para que possamos fazer a análise explorativa dos dados, a partir da data, será necessário converter a data que está em string para *datatime*

In [28]:
base_beer['Data'] = pd.to_datetime(base_beer['Data'])
base_beer['Mes'] = base_beer['Data'].dt.month 
base_beer['Mes str'] = base_beer['Mes'].astype(str).replace({'1': 'Janeiro','2': 'Fevereiro','3': 'Março','4': 'Abril','5': 'Maio','6': 'Junho','7': 'Julho'})
base_beer['Dia'] = base_beer['Data'].dt.dayofweek 
base_beer['Dia'] = base_beer['Dia'].astype(str).replace({'0': 'Seg','1': 'Ter','2': 'Qua','3': 'Qui','4': 'Sex','5': 'Sáb','6': 'Dom'})

Uma vez que já identificamos o dia da semana e mes, agora será necessário identificar os feriados, para isso, localizei todos os feriados nacionais e locais da cidade neste respectivo site https://www.feriados.com.br/2015

In [29]:
def acharferiado(registro):
    if registro['Data'] == pd.to_datetime('2015-01-01'):
      return 1
    elif registro['Data']== pd.to_datetime('2015-01-25'):
      return 1
    elif registro['Data']== pd.to_datetime('2015-02-16'):
      return 1
    elif registro['Data']== pd.to_datetime('2015-01-17'):
      return 1
    elif registro['Data']== pd.to_datetime('2015-01-18'):
      return 1
    elif registro['Data']== pd.to_datetime('2015-04-03'):
      return 1
    elif registro['Data']== pd.to_datetime('2015-04-21'):
      return 1
    elif registro['Data']== pd.to_datetime('2015-05-01'):
      return 1
    elif registro['Data']== pd.to_datetime('2015-06-04'):
      return 1
    elif registro['Data']== pd.to_datetime('2015-07-09'):
      return 1
    elif registro['Data']== pd.to_datetime('2015-09-07'):
      return 1
    elif registro['Data']== pd.to_datetime('2015-10-12'):
      return 1
    elif registro['Data']== pd.to_datetime('2015-10-15'):
      return 1
    elif registro['Data']== pd.to_datetime('2015-10-28'):
      return 1
    elif registro['Data']== pd.to_datetime('2015-11-02'):
      return 1
    elif registro['Data']== pd.to_datetime('2015-11-15'):
      return 1
    elif registro['Data']== pd.to_datetime('2015-11-20'):
      return 1
    elif registro['Data']== pd.to_datetime('2015-12-25'):
      return 1
    else:
      return 0
base_beer['Feriado'] = base_beer.apply(acharferiado, axis=1)


## 1. Qual dia da semana houve um consumo maior de cerveja?

In [30]:
consumo_dia = ['Dia', 'Consumo de cerveja (litros)']
diaconsumo = base_beer[consumo_dia].groupby('Dia', sort= True).sum()
diaconsumo#.plot(kind='bar', legend=True);


,Consumo de cerveja (litros)
Dia,
Dom,1508.391
Qua,1238.140
Qui,1276.057
Seg,1227.701
Sex,1255.170
Sáb,1499.572
Ter,1266.468


Para sabermos qual dia da semana em que se consome mais cerveja na cidade de SP, foi necessário somar o consumo diário, com isso podemos ver que o dia em que mais se consome cerveja, é aos finais de semana, sábado e domingo

## 2. Qual data houve um consumo maior

In [31]:
base_beer[base_beer['Consumo de cerveja (litros)']==base_beer['Consumo de cerveja (litros)'].max()]

,Data,Temperatura Media (C),Temperatura Minima (C),Temperatura Maxima (C),Precipitacao (mm),Final de Semana,Consumo de cerveja (litros),Mes,Mes str,Dia,Feriado
9,2015-01-10,"26,76","22,1","34,2",0,1.0,37.937,1,Janeiro,Sáb,0


A data em que houve um maior consumo de cerveja ocorreu na data 10/01/2015. Se observarmos, esta data era um final de semana, no caso no sábado, e no mês de janeiro. Como já analisado, foram onde teve um maior consumo.

## 3. Qual data houve um consumo menor?

In [32]:
base_beer[base_beer['Consumo de cerveja (litros)']==base_beer['Consumo de cerveja (litros)'].min()]

,Data,Temperatura Media (C),Temperatura Minima (C),Temperatura Maxima (C),Precipitacao (mm),Final de Semana,Consumo de cerveja (litros),Mes,Mes str,Dia,Feriado
202,2015-07-22,"14,52","12,8","16,1",0,0.0,14.343,7,Julho,Qua,0


O dia de menor consumo, foi na data 22/07/2015, e num dia de muito frio

## 4. Em qual mês houve um consumo maior ?


In [33]:
consumo_mes= ['Mes', 'Consumo de cerveja (litros)']
mesconsumo = base_beer[consumo_mes].groupby('Mes', sort= True).sum()
mesconsumo#.plot(kind='barh');

,Consumo de cerveja (litros)
Mes,
1,889.007
2,718.848
3,776.128
4,738.723
5,706.750
6,708.812
7,699.759
8,803.178
9,755.944


## 5. Era um dia útil, final de semana ou feriado quando ocorreu um consumo maior?


o dataset possui dados do dia 1 de janeiro ao dia 31 de dezembro de 2015 e de acordo com os dados extraídos, o maior consumo ocorreu aos finais de semana e no mês de janeiro.

Para este dataset, irei executar o modelo de regressão linear, só que para isto irei usar a classe runregression, esta classe irá permitir que possamos utilizar o cross validation e guarda as métricas (MAE, MSE, RMSE e R-quadrado) de cada fold e ao fim, apresenta a média e o desvio padrão.

Créditos a minha orientadora maravilhosa [Laura Emmanuela](https://github.com/lauraemmanuella/AprendizadoMaquina/blob/main/9_Regress%C3%A3o.ipynb)

In [34]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn import metrics

class runregression:
  '''
    Parâmetros de entrada:

    X: atributos de entradas
    y: atributo alvo
    model: algoritmo para construção do estimador
    cv: quantidade de folds da validação cruzada

    Guarda as métricas de cada execução em resultados e 
    Mostra os valores médios com desvio padrão para as métricas (mostraresultadomedio)   
  '''

  def __init__(self, X, y, model, cv = 5):
    
    self.__resultados = {
      'MAE': [],
      'MSE': [],
      'R2': []
    }

    print(f'validação cruzada com {cv} folds')
    self.__avaliamodelo_cv(X, y, model, cv)

  @property
  def resultados(self):
    return self.__resultados

  def __avaliamodelo_cv(self, X, y, model, cv):
    #gera as amostras para cv folds com embaralhamento (permite fazer várias validações cruzadas)
    kf = KFold(shuffle=True, n_splits=cv)
    #para cada fold: treina, testa e armazena os resultados
    for train_index, test_index in kf.split(X, y):
      model.fit(X[train_index], y[train_index])
      pred = model.predict(X[test_index])
      self.__resultados['MAE'].append(metrics.mean_absolute_error(y[test_index], pred))
      self.__resultados['MSE'].append(metrics.mean_squared_error(y[test_index], pred))
      self.__resultados['R2'].append(metrics.r2_score(y[test_index], pred))

  def mostraresultadomedio(self):
    print(f"MAE: {np.mean(self.__resultados['MAE']):.2f} +- {np.std(self.__resultados['MAE']):.2f}")
    print(f"MSE: {np.mean(self.__resultados['MSE']):.2f} +- {np.std(self.__resultados['MSE']):.2f}")
    print(f"R2: {np.mean(self.__resultados['R2']):.2f} +- {np.std(self.__resultados['R2']):.2f}")

In [35]:
lista_temp = ['Temperatura Media (C)', 'Temperatura Minima (C)','Temperatura Maxima (C)', 'Precipitacao (mm)']
base_beer[lista_temp] = base_beer[lista_temp].apply(lambda x: x.str.replace(',', '.'))

X = base_beer.drop(['Data','Mes','Mes str','Dia','Consumo de cerveja (litros)', 'Feriado'], axis=1)
y = base_beer['Consumo de cerveja (litros)']

X = np.array(X)
y = np.array(y)

normaliza = StandardScaler()
X = normaliza.fit_transform(X)
#


Usando a classe runregression

In [36]:
cross_validation_RL = runregression(X, y, LinearRegression(), cv = 10)
cross_validation_RL.mostraresultadomedio()

validação cruzada com 10 folds
MAE: 2.00 +- 0.19
MSE: 5.59 +- 0.92
R2: 0.70 +- 0.06


Para avaliar os resultados obtidos, iremos utilizar as seguintes métricas:

* **Erro Médio Absoluto:** O erro médio absoluto, conhecido como MAE, é a diferença entre os valores reais e os valores previstos pelo modelo. Quanto menor este valor do MAE, melhor o desempenho deste modelo. Um erro que apresente um valor igual ou próximo a zero significa que o o modelo é muito bom. Para este modelo, o MAE deu 2.00, o que é bom pois está muito próximo a 0.

* **Erro Quadrático Médio:** É calculado tomando a média do quadrado  da diferença entre os valores reais e os valores previsto pelo modelo. 

* **R-quadrado:** O r-quadrado, representa o coeficiente de determinação, ele nos diz quantos pontos caem na reta da regressão. O valor de 0.70, indica que 70% dos dados se ajustam ao modelo de regressão

